<a href="https://colab.research.google.com/github/Todah01/Summoners-War-Analysis/blob/main/GuildMemberAttackLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
from collections import defaultdict
import pickle
import io
import pandas as pd
import json
import sys
import re

attack_logs = []

# uploaded = files.upload()

# df = pd.read_json(io.StringIO(uploaded['siege_replay_JP.json'].decode('utf-8')))
# df.head()

with open('GuildSiegeBattleLog_1.json', 'r', encoding='UTF8') as f:
    attack_log_1 = json.load(f)
    attack_logs.append(attack_log_1)

# with open('GuildSiegeBattleLog_2.json', 'r', encoding='UTF8') as f:
#     attack_log_2 = json.load(f)
#     attack_logs.append(attack_log_2)

# with open('GuildSiegeBattleLog_3.json', 'r', encoding='UTF8') as f:
#     attack_log_3 = json.load(f)
    # attack_logs.append(attack_log_3)

# with open('GuildSiegeBattleLog_defense.json', 'r', encoding='UTF8') as f:
#     defense_log = json.load(f)

with open('js_monsters_kr.json', 'r', encoding='UTF8') as f:
    monsters = json.load(f)

# for idx in range(1, len(replay_json)):
#   print(replay_json[idx]['data'])

guild_member_list = ['#개복치영다','#고니','#꽃새우','#람릿','#매봉','#맥주','#미실','#사이타마','#삼겹살','#쑹싸쑹','#연어와곰','요판','#요한','#웅블랑','#죠죠','#주나','#차차','#해벌레','#현식이',
                     '호우도르','#호잉','#흐꾸','#히릿','#BANANA','#UnknownBird']

print(len(attack_logs))
print(len(guild_member_list))

1
25


In [2]:
for attack_log in attack_logs:
  for idx in range(len(attack_log['log_list'])):
    for info in attack_log['log_list'][idx]['guild_info_list']:
      print(info['siege_id'], info['guild_name'])
    print('\n')

2023120201 BEEP
2023120201 사춘기
2023120201 #Hash tag




In [16]:
member_record_dict = defaultdict(lambda:defaultdict(int))
member_log = attack_log['log_list'][0]['battle_log_list']
# [monsters[str(monster)] for monster in sorted(info['view_battle_deck_info'][1][1:], reverse = True)]

for idx in range(len(member_log)):
  if member_log[idx]['win_lose'] == 1:
    member_record_dict[member_log[idx]['wizard_name']]['win'] += 1
  elif member_log[idx]['win_lose'] == 2:
    member_record_dict[member_log[idx]['wizard_name']]['lose'] += 1
  else:
    print(member_log[idx]['wizard_name'], member_log[idx]['win_lose'])

if len(member_record_dict) < 25:
  print('error')

In [20]:
member_elaborate_list = []
for key in member_record_dict:
  temp_member_list = []
  temp_member_list.append(key)
  temp_member_list.append(member_record_dict[key]['win'])
  temp_member_list.append(member_record_dict[key]['lose'])
  member_elaborate_list.append(temp_member_list)

In [8]:
def get_data_list_from_log(logs, number = 0):
  data_list = []
  for idx in range(len(logs['log_list']) - number):
    for info in logs['log_list'][idx]['battle_log_list']:
      # 1 == win / 2 == lose
      temp_list = []
      for defense_monster_data in info['view_battle_deck_info'][1]:
        temp_list.append(monsters[str(defense_monster_data)])
      for attack_monster_data in info['view_battle_deck_info'][0]:
        temp_list.append(monsters[str(attack_monster_data)])

      temp_list.append(info['win_lose'])

      data_list.append(temp_list)

  return data_list

In [9]:
def make_dict_from_data_list(def_vs_atk_dict, logs, number = 0):
  for idx in range(len(logs['log_list']) - number):
    for info in logs['log_list'][idx]['battle_log_list']:
      # 1 == win / 2 == lose
      temp_list = []
      def_key_list = []
      def_key = ','.join([monsters[str(info['view_battle_deck_info'][1][0])]] + [monsters[str(monster)] for monster in sorted(info['view_battle_deck_info'][1][1:], reverse = True)])
      atk_key = ','.join([monsters[str(info['view_battle_deck_info'][0][0])]] + [monsters[str(monster)] for monster in sorted(info['view_battle_deck_info'][0][1:], reverse = True)])

      def_vs_atk_dict[def_key][atk_key]['total'] += 1
      if info['win_lose'] == 1:
        def_vs_atk_dict[def_key][atk_key]['win'] += 1
      else:
        def_vs_atk_dict[def_key][atk_key]['lose'] += 1

In [10]:
def check_data_loss(data_list, data_dict):
  length_of_list = 0
  length_of_dict = 0

  for data in data_list:
    length_of_list += len(data)

  for key in data_dict:
    for member in data_dict[key]:
      length_of_dict += data_dict[key][member]['total']

  return length_of_list == length_of_dict

In [11]:
data_list = []
for idx in range(len(attack_logs)):
  n = 0
  if idx == 1:
    n = 1
  data_list.append(get_data_list_from_log(attack_logs[idx], n))

In [12]:
def_vs_atk_dict = defaultdict(lambda:defaultdict(lambda:defaultdict(int)))
for idx in range(len(attack_logs)):
  n = 0
  if idx == 1:
    n = 1
  make_dict_from_data_list(def_vs_atk_dict, attack_logs[idx], n)

In [13]:
print(check_data_loss(data_list, def_vs_atk_dict))

True


In [14]:
# with open('data_dict.pkl', 'wb') as f:
#   pickle.dump(def_vs_atk_dict, f)

In [15]:
elaborate_list = []
for key in def_vs_atk_dict:
  temp_key = key.split(',')

  for member in def_vs_atk_dict[key]:
    temp_member_list = temp_key[:]
    temp_member = member.split(',')
    for monster in temp_member:
      temp_member_list.append(monster)

    temp_member_list.append(def_vs_atk_dict[key][member]['total'])

    if 'win' in def_vs_atk_dict[key][member]:
      temp_member_list.append(def_vs_atk_dict[key][member]['win'])
    else:
      temp_member_list.append(0)

    if 'lose' in def_vs_atk_dict[key][member]:
      temp_member_list.append(def_vs_atk_dict[key][member]['lose'])
    else:
      temp_member_list.append(0)

    elaborate_list.append(temp_member_list)

In [22]:
df_member_log_elaborate = pd.DataFrame(member_elaborate_list, columns=["User", "Win", "Lose"])
df_member_log_elaborate["odds_of_winning"] = round((df_member_log_elaborate["Win"] / (df_member_log_elaborate["Win"] + df_member_log_elaborate["Lose"])) * 100, 2)
sorted_df = df_member_log_elaborate.sort_values(by=["User"], ascending=[True])
sorted_df.to_csv('df_member_log.csv', index=False, encoding='CP949')
files.download('df_member_log.csv')

df_attack_log_elaborate = pd.DataFrame(elaborate_list, columns=["def_monster1", "def_monster2", "def_monster3", "atk_monster1", "atk_monster2", "atk_monster3", "Total", "Win", "Lose"])
df_attack_log_elaborate["odds_of_winning"] = round((df_attack_log_elaborate["Win"] / df_attack_log_elaborate["Total"]) * 100, 2)
sorted_df = df_attack_log_elaborate.sort_values(by=["def_monster1", "def_monster2", "def_monster3", "Total"], ascending=[True, True, True, False])
sorted_df.to_csv('df_attack_log.csv', index=False, encoding='CP949')
files.download('df_attack_log.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>